# Quant Finance - Clase 2

In [1]:
### Importo los modulos

import sys
sys.path.append('..')

from Codigo.data_nyse import *

import yfinance as yf
import datetime

ModuleNotFoundError: No module named 'Codigo.data_nyse'

In [23]:
panel_opciones_FB = obtener_panel_opciones_nyse('FB', True)


In [38]:
panel_opciones_FB['PrecioAvg'] = (panel_opciones_FB.Bid + panel_opciones_FB.Ask)/2
panel_opciones_FB_calls = panel_opciones_FB[panel_opciones_FB.CallPut=='C']
panel_opciones_FB_puts = panel_opciones_FB[panel_opciones_FB.CallPut=='P']

panel_opciones_FB_reg = panel_opciones_FB_calls.copy()

panel_opciones_FB_reg['Put_menos_Call'] = 0.0


In [62]:
print(set(panel_opciones_FB_reg.TTM.values))


{290, 3, 38, 199, 136, 745, 10, 171, 108, 45, 654, 80, 17, 24, 507, 318, 31}


In [ ]:
for idx in panel_opciones_FB_reg.index:
    try:
        panel_opciones_FB_reg['Put_menos_Call'].values[idx] = panel_opciones_FB_puts[(panel_opciones_FB_puts.Strike == panel_opciones_FB_reg.Strike.values[idx]) & (panel_opciones_FB_puts.TTM == panel_opciones_FB_reg.TTM.values[idx])].PrecioAvg.values[0] - panel_opciones_FB_reg['PrecioAvg'].values[idx]
    except:
        panel_opciones_FB_reg['Put_menos_Call'].values[idx] = -99.9
            

panel_opciones_FB_reg = panel_opciones_FB_reg.dropna()

panel_opciones_FB_reg

En el caso de una Europea con tasa de dividendos div: $$C(0)=P(0) + S(0)e^{-divT} - K e^{-rT}$$ 



In [64]:
import statsmodels.api as sm
import pandas as pd
3
panel_opciones_FB_reg_TTM =  panel_opciones_FB_reg[panel_opciones_FB_reg.TTM==mat]

X = panel_opciones_FB_reg_TTM[['Strike','Spot']] # here we have 2 variables for multiple regression. If you just want to use one variable for simple linear regression, then use X = df['Interest_Rate'] for example.Alternatively, you may add additional variables within the brackets
Y = panel_opciones_FB_reg_TTM['Put_menos_Call']
 
# with sklearn
regr = linear_model.LinearRegression()
regr.fit(X, Y)

model = sm.OLS(Y, X).fit()
predictions = model.predict(X) 
 
print_model = model.summary()
#print(print_model)


a = model.params[0]
b = - model.params[1]

import math
r = math.log(1/a)/(mat/365)
div = math.log(1/b)/(mat/365)


print(a)

print(div)

#a=exp(-rT) 
#b = exp(-divT)

#la P-C parity queda lineal en a y b:

#P-C = K a - b S 

#Missing['Price']=np.where(
#    Missing['MissingOption']== 'C', 
#    Missing['Spot']*np.exp(-div*Missing['TTM']/365)-Missing['Strike']*np.exp(-r*Missing['TTM']/365)+Missing['Last'],
#    Missing['Strike']*np.exp(-r*Missing['TTM']/365)-Missing['Spot']*np.exp(-div*Missing['TTM']/365)+Missing['Last'])

0.25275104877458954
1.037842203208205


## Ploteando los datos

### Para cada vencimiento, grafico los strikes

In [ ]:


TTMs = list(set(panel_opciones_ggal_nyse_limpio.TTM.values))
TTMs

In [ ]:
TTM = 23
panel_opciones_ggal_nyse_limpio[(panel_opciones_ggal_nyse_limpio.TTM == TTM) & (panel_opciones_ggal_nyse_limpio.CallPut == "C")].head(10)

In [ ]:
from matplotlib import pyplot as plt
CP = "C"
for TTM in TTMs:
    Ks = panel_opciones_ggal_nyse_limpio[(panel_opciones_ggal_nyse_limpio.TTM == TTM) & (panel_opciones_ggal_nyse_limpio.CallPut == CP)].Strike.values
    precios = panel_opciones_ggal_nyse_limpio[(panel_opciones_ggal_nyse_limpio.TTM == TTM) & (panel_opciones_ggal_nyse_limpio.CallPut == CP)].Last.values
    ## Last VS (B+A)/2
    plt.plot(Ks, precios,'*', label='TTM = '+str(TTM))


plt.legend()

plt.xlabel('Strike')
plt.ylabel('Precio de mercado del Call')
plt.title('Precio de mercado de las opciones Call con respecto al Strike')
    
plt.show()
    

In [ ]:

CP = "P"
for TTM in TTMs:
    Ks = panel_opciones_ggal_nyse_limpio[(panel_opciones_ggal_nyse_limpio.TTM == TTM) & (panel_opciones_ggal_nyse_limpio.CallPut == CP)].Strike.values
    precios = panel_opciones_ggal_nyse_limpio[(panel_opciones_ggal_nyse_limpio.TTM == TTM) & (panel_opciones_ggal_nyse_limpio.CallPut == CP)].Last.values
    plt.plot(Ks, precios,'*', label='TTM = '+str(TTM))


plt.legend()

plt.xlabel('Strike')
plt.ylabel('Precio de mercado del Call')
plt.title('Precio de mercado de las opciones Call con respecto al Strike')
    
plt.show()

### Para cada strike, grafico los TTM

In [ ]:
Ks = list(set(panel_opciones_ggal_nyse_limpio.Strike.values))

In [ ]:
from matplotlib import pyplot as plt
CP = "C"
for K in Ks:
    TTMs = panel_opciones_ggal_nyse_limpio[(panel_opciones_ggal_nyse_limpio.Strike == K) & (panel_opciones_ggal_nyse_limpio.CallPut == CP)].Strike.values
    precios = panel_opciones_ggal_nyse_limpio[(panel_opciones_ggal_nyse_limpio.Strike == K) & (panel_opciones_ggal_nyse_limpio.CallPut == CP)].Last.values
    if len(TTMs)>1:
        plt.plot(TTMs, precios,'*', label='Strike = '+str(K))

plt.legend()

plt.xlabel('TTM')
plt.ylabel('Precio de mercado del Call')
plt.title('Precio de mercado de las opciones Call con respecto al TTM')
    
plt.show()
    

## Ploteando los datos con un ticker mega liquido

In [ ]:
# NYSE AAPL
ticker = 'AAPL'
panel_opciones_nyse_limpio = obtener_panel_opciones_nyse(ticker, True)

TTMs = list(set(panel_opciones_nyse_limpio.TTM.values))
TTMs.sort()
TTMs

In [ ]:

from matplotlib import pyplot as plt
CP = "C"
for TTM in TTMs[0:5]:
    Ks = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.TTM == TTM) & (panel_opciones_nyse_limpio.CallPut == CP)].Strike.values
    precios = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.TTM == TTM) & (panel_opciones_nyse_limpio.CallPut == CP)].Last.values
    plt.plot(Ks, precios, '.',label='TTM = '+str(TTM))


plt.legend()

plt.xlabel('Strike')
plt.ylabel('Precio de mercado del Call')
plt.title('Precio de mercado de las opciones Call con respecto al Strike')
    
plt.show()
    

In [ ]:

from matplotlib import pyplot as plt
CP = "C"
for TTM in TTMs[5:11]:
    Ks = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.TTM == TTM) & (panel_opciones_nyse_limpio.CallPut == CP)].Strike.values
    precios = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.TTM == TTM) & (panel_opciones_nyse_limpio.CallPut == CP)].Last.values
    plt.plot(Ks, precios, '.',label='TTM = '+str(TTM))


plt.legend()

plt.xlabel('Strike')
plt.ylabel('Precio de mercado del Call')
plt.title('Precio de mercado de las opciones Call con respecto al Strike')
    
plt.show()
    

In [ ]:

from matplotlib import pyplot as plt
CP = "C"
for TTM in TTMs[11:17]:
    Ks = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.TTM == TTM) & (panel_opciones_nyse_limpio.CallPut == CP)].Strike.values
    precios = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.TTM == TTM) & (panel_opciones_nyse_limpio.CallPut == CP)].Last.values
    plt.plot(Ks, precios,'.',label='TTM = '+str(TTM))


plt.legend()

plt.xlabel('Strike')
plt.ylabel('Precio de mercado del Call')
plt.title('Precio de mercado de las opciones Call con respecto al Strike')
    
plt.show()
    

In [ ]:
from matplotlib import pyplot as plt
CP = "P"
for TTM in TTMs[0:5]:
    Ks = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.TTM == TTM) & (panel_opciones_nyse_limpio.CallPut == CP)].Strike.values
    precios = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.TTM == TTM) & (panel_opciones_nyse_limpio.CallPut == CP)].Last.values
    plt.plot(Ks, precios, '.',label='TTM = '+str(TTM))


plt.legend()

plt.xlabel('Strike')
plt.ylabel('Precio de mercado del Put')
plt.title('Precio de mercado de las opciones Put con respecto al Strike')
    
plt.show()
    

In [ ]:
from matplotlib import pyplot as plt
CP = "P"
for TTM in TTMs[5:11]:
    Ks = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.TTM == TTM) & (panel_opciones_nyse_limpio.CallPut == CP)].Strike.values
    precios = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.TTM == TTM) & (panel_opciones_nyse_limpio.CallPut == CP)].Last.values
    plt.plot(Ks, precios, '.',label='TTM = '+str(TTM))


plt.legend()

plt.xlabel('Strike')
plt.ylabel('Precio de mercado del Put')
plt.title('Precio de mercado de las opciones Put con respecto al Strike')
    
plt.show()
    

In [ ]:
from matplotlib import pyplot as plt
CP = "P"
for TTM in TTMs[11:17]:
    Ks = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.TTM == TTM) & (panel_opciones_nyse_limpio.CallPut == CP)].Strike.values
    precios = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.TTM == TTM) & (panel_opciones_nyse_limpio.CallPut == CP)].Last.values
    plt.plot(Ks, precios, '.',label='TTM = '+str(TTM))


plt.legend()

plt.xlabel('Strike')
plt.ylabel('Precio de mercado del Put')
plt.title('Precio de mercado de las opciones Put con respecto al Strike')
    
plt.show()
    

In [ ]:
Ks = list(set(panel_opciones_nyse_limpio.Strike.values))
Ks.sort()
len(Ks)

In [ ]:
from matplotlib import pyplot as plt
CP = "C"
for K in Ks[45:55]:
    TTMs = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.Strike == K) & (panel_opciones_nyse_limpio.CallPut == CP)].Strike.values
    precios = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.Strike == K) & (panel_opciones_nyse_limpio.CallPut == CP)].Last.values
    if len(TTMs)>1:
        plt.plot(TTMs, precios,'*', label='Strike = '+str(K))

plt.legend()

plt.xlabel('TTM')
plt.ylabel('Precio de mercado del Call')
plt.title('Precio de mercado de las opciones Call con respecto al TTM')
    
plt.show()

In [ ]:
from matplotlib import pyplot as plt
CP = "P"
for K in Ks[100:]:
    TTMs = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.Strike == K) & (panel_opciones_nyse_limpio.CallPut == CP)].Strike.values
    precios = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.Strike == K) & (panel_opciones_nyse_limpio.CallPut == CP)].Last.values
    if len(TTMs)>1:
        plt.plot(TTMs, precios,'*', label='Strike = '+str(K))

plt.legend()

plt.xlabel('TTM')
plt.ylabel('Precio de mercado del Put')
plt.title('Precio de mercado de las opciones Put con respecto al TTM')
    
plt.show()

## Pasar a Excel

In [ ]:
panel_opciones_ggal_nyse_crudo = obtener_panel_opciones_nyse('GGAL', False)
panel_opciones_ggal_nyse_crudo.to_excel('Clase_2_panel_opciones_ggal_nyse_crudo.xls')

panel_opciones_ggal_nyse_limpio = obtener_panel_opciones_nyse('GGAL', True)
panel_opciones_ggal_nyse_limpio.to_excel('Clase_2_panel_opciones_ggal_nyse_limpio.xls')


panel_opciones_aapl_nyse_limpio = obtener_panel_opciones_nyse('AAPL', True)
panel_opciones_aapl_nyse_limpio.to_excel('Clase_2_panel_opciones_aapl_nyse_limpio.xls')                                                
         

## Extras vistas en clase

In [ ]:
TTMs = list(set(panel_opciones_nyse_limpio.TTM.values))
TTMs.sort()


### Plot de Volatilidades Implicitas dadas por Yahoo Finances

In [ ]:

from matplotlib import pyplot as plt
CP = "C"
for TTM in TTMs[0:3]:
    Ks = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.TTM == TTM) & (panel_opciones_nyse_limpio.CallPut == CP)].Strike.values
    iv = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.TTM == TTM) & (panel_opciones_nyse_limpio.CallPut == CP)].impliedVolatility.values
    plt.plot(Ks, iv, '.',label='TTM = '+str(TTM))


plt.legend()

plt.xlabel('Strike')
plt.ylabel('IV')
plt.title('IV de yahoo de las opciones Call con respecto al strike')
    
plt.show()

### Usar en vez de LAST, (BID+ASK) /2

In [ ]:
panel_opciones_nyse_limpio['Avg']=(panel_opciones_nyse_limpio.Bid+panel_opciones_nyse_limpio.Ask)/2

In [ ]:

from matplotlib import pyplot as plt
CP = "C"
for TTM in TTMs[0:1]:
    Ks = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.TTM == TTM) & (panel_opciones_nyse_limpio.CallPut == CP)].Strike.values
    precios_L = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.TTM == TTM) & (panel_opciones_nyse_limpio.CallPut == CP)].Last.values
    precios_A = panel_opciones_nyse_limpio[(panel_opciones_nyse_limpio.TTM == TTM) & (panel_opciones_nyse_limpio.CallPut == CP)].Avg.values
    plt.plot(Ks, precios_L, '.',label='LAST TTM = '+str(TTM))
    plt.plot(Ks, precios_A, '*',label='AVG TTM = '+str(TTM))


plt.legend()

plt.xlabel('Strike')
plt.ylabel('Precio de mercado del Call')
plt.title('Precio de mercado de las opciones Call con respecto al Strike')
    
plt.show()